In [1]:
# Use autoplot 207's code for this app
import sys
from datetime import datetime

import numpy as np
from pyproj import Transformer

import geopandas as gpd
import pandas as pd
from iemweb.autoplot.scripts200.p207 import USEME, add_zeros, compute_grid_bounds, do_analysis, load_data
from matplotlib.patches import Rectangle
from pyiem.nws.vtec import NWS_COLORS
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_sqlalchemy_conn, utc
from shapely.geometry import Point

In [2]:
STORM_NUMBER = 12
WINTER = "2024-2025"
TITLE = "11-12 February 2025"
SUBTITLE = "8 AM 13 February 2025"
SETPOINT_LOCS = {}
# naive US Central local time
sts = datetime(2025, 2, 12, 10)
ets = datetime(2025, 2, 13, 19)
# Get available data
ctx = {
    "coop": "yes",
    "cocorahs": "yes",
    "t": "state",
    "sz": 30,
    "z": "yes",
    "f": "linear",
    "v": "snow",
    "wfo": "DMX",
}
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, "IA")
df = load_data(ctx, sts, ets)
df = df[~df["nwsli"].isin(["DSXI4", "DMX"])]
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
            """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
            conn,
            params=(sts - datetime.timedelta(days=1), ets),
        )
    print(df[df["state"] == "IA"])
    mp.plot_values(
        df.lon.values,
        df.lat.values,
        df.magnitude.values,
        fmt="%.2f",
        labelbuffer=1,
        color="purple",
    )


def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE,),
        subtitle=(
            f"Snowfall totals till {SUBTITLE} from NWS COOP, LSR, CoCoRaHS Reports; "
            f"IEM {WINTER} Winter Storm #{STORM_NUMBER}"
        ),
        twitter=True,
    )
    cmap = nwssnow()
    # cmap = get_cmap("Greens")
    ramp = [0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]
    # ramp = [0.1, 1, 2, 3, 4]
    mp.contourf(lons, lats, vals, np.array(ramp), cmap=cmap, clip_on=True)
    df_useme_plot = df2[(df2["val"] >= lower) & (df2["val"] < upper)]
    print(df[df["state"] == "IA"])
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(
            df_useme_plot["lon"],
            df_useme_plot["lat"],
            df_useme_plot["val"].values,
            "%s",
            labels=df_useme_plot["nwsli"].values,
            textsize=10,
            labeltextsize=10,
            labelbuffer=1,
        )
    return mp

In [4]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(
            SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1]
        )
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True


def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.0
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz / 2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz / 2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1


def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets),
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0,), ("Snow Squall Warning",), loc=1).set_zorder(
        1000
    )

In [5]:
df

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
0,NY,BUF,13.0,-75.940000,43.240000,POINT (1913953.476 90485.474),True,0,True,LSR,55,21
1,NY,BUF,10.1,-76.540000,43.450000,POINT (1861349.872 99080.037),True,1,True,LSR,53,21
2,NY,BGM,9.1,-75.260000,43.240000,POINT (1965963.745 106829.718),True,2,True,LSR,56,21
3,SD,FSD,6.9,-96.830000,43.440000,POINT (255882.951 -168486.12),True,3,True,LSR,0,12
4,NY,BUF,5.8,-76.060000,43.300000,POINT (1902844.007 94109.228),True,4,True,LSR,54,21
...,...,...,...,...,...,...,...,...,...,...,...,...
992,IL,DVN,0.0,-90.745164,40.462365,POINT (781548.22 -460586.388),True,IL-MCD-7,True,COCORAHS,17,2
993,IL,ILX,0.0,-90.502290,40.153654,POINT (805717.348 -492401.3),True,IL-SY-6,True,COCORAHS,18,1
994,IL,LSX,0.0,-91.310800,39.912501,POINT (740141.256 -526640.678),True,IL-AD-16,True,COCORAHS,15,0
995,Z,NaN,0.0,-94.702760,41.608650,POINT (440227.691 -362858.648),True,Z1,False,NaN,6,6


In [21]:
def main():
    setpoints = [

    ]
    #if setpoints:
    #    add_setpoints(setpoints)
    cull = [
        743,
        'ALGI4',
        'SACI4',
        'WTRI4',
        'IA-DV-10',
        409,
        166,
        156,
        58,
    ]
    if cull:
        df.loc[df["nwsli"].isin(cull), USEME] = False
    hardcode = [
        ('SHDI4', 0.5),
        ('MZUI4', 7.5),
    ]
    for nwsli, val in hardcode:
        df.loc[df["nwsli"] == nwsli, "val"] = val

    ctx["csector"] = "IA"
    mp = workflow(ctx, df, isfinal=True, lower=0., upper=10)
    #draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename="250213.png")
    mp.close()


main()

     state  wfo      val        lon        lat  \
0       IA  DVN  11.7000 -90.920000  41.870000   
3       IA  DMX  10.0000 -93.960000  40.840000   
4       IA  DMX  10.0000 -93.240000  41.230000   
7       IA  DMX   9.5000 -92.920000  41.410000   
9       IA  DVN   9.1000 -91.610000  42.060000   
...    ...  ...      ...        ...        ...   
1723    IA  DMX   0.6000 -94.554029  42.963132   
1729    IA  FSD   0.5000 -95.605553  42.399473   
1741    IA  ARX   0.2000 -92.320374  43.055346   
1742    IA  FSD   0.0001 -96.150450  42.784900   
1744    IA  DMX   0.0000 -92.903614  41.127947   

                                 geo  used_for_analysis    nwsli  plotme  \
0     POINT (750222.465 -306449.653)               True        0    True   
3     POINT (507875.792 -443866.753)               True        3    True   
4     POINT (564831.511 -395928.802)               True        4    True   
7     POINT (589826.182 -373727.429)               True        7    True   
9     POINT (691371

## 